### 1. 필요 라이브러리 호출

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import json
from bs4 import BeautifulSoup

### 2. 데이터 불러오기

In [2]:
df_hrd = pd.read_csv('hrd_net.csv',index_col=0)

### 3. OPEN API 활용 데이터 수집 - 훈련과정 별 상세 정보

In [3]:
trpr_id = df_hrd['훈련과정 ID'].unique().tolist()

In [4]:
dic = {}

for i in tqdm(range(len(df_hrd))):


    if df_hrd.loc[i,'훈련과정 ID'] in dic:

        continue

    else:

        dic[df_hrd.loc[i,'훈련과정 ID']] = df_hrd.loc[i,'훈련기관 ID']

100%|██████████| 88500/88500 [00:05<00:00, 17243.68it/s]


In [5]:
columns = ['trprId', 'trprDegr', 'trprGbn', 'trprTarget', 'trprTargetNm', 'trprNm', 'inoNm', 'instIno', 'traingMthCd', 'trprChap', 'trprChapTel', 'trprChapEmail', 'ncsYn', 'ncsCd', 'ncsNm', 'trDcnt', 'trtm', 'nonNcsCourseTheoryTime', 'nonNcsCoursePrcttqTime', 'zipCd', 'addr1', 'addr2', 'hpAddr', 'filePath', 'pFileName', 'torgParGrad', 'perTrco', 'instPerTrco']

#### OPEN API 제한 초과로 나눠서 수집 후 CONCAT

In [12]:
trpr_len = df_hrd['훈련과정 ID'].value_counts()

df_base = pd.DataFrame(columns=columns) 


for tp in tqdm(trpr_id): # 19960개

    n = trpr_len[tp] # 한 훈련과정 당 회차가 279, 2 뭐 다양해

    url = 'https://www.hrd.go.kr/jsp/HRDP/HRDPO00/HRDPOA60/HRDPOA60_2.jsp?authKey=BVnnOlx3MTUef0PaPC4O6IHKCsQ3uX9L&returnType=JSON&outType=2&srchTrprId={}&srchTrprDegr={}&srchTorgId={}'.format(tp,n,dic[tp])

    response = requests.get(url)

    js = response.json()

    base_dic = pd.DataFrame.from_dict([json.loads(js['returnJSON'])['inst_base_info']])
    
    df_base = pd.concat([df_base, base_dic])

 84%|████████▍ | 16718/19960 [1:25:33<16:35,  3.26it/s]  


ConnectTimeout: HTTPSConnectionPool(host='www.hrd.go.kr', port=443): Max retries exceeded with url: /jsp/HRDP/HRDPO00/HRDPOA60/HRDPOA60_2.jsp?authKey=BVnnOlx3MTUef0PaPC4O6IHKCsQ3uX9L&returnType=JSON&outType=2&srchTrprId=AIG20220000407579&srchTrprDegr=2&srchTorgId=500040680393 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000024A51B43700>, 'Connection to www.hrd.go.kr timed out. (connect timeout=None)'))

In [25]:
temp = trpr_id[16718:]

In [30]:
trpr_len = df_hrd['훈련과정 ID'].value_counts()

df_base_from16718 = pd.DataFrame(columns=columns) 


for tp in tqdm(temp):

    n = trpr_len[tp] # 한 훈련과정 당 회차가 279, 2 뭐 다양해

    url = 'https://www.hrd.go.kr/jsp/HRDP/HRDPO00/HRDPOA60/HRDPOA60_2.jsp?authKey=BVnnOlx3MTUef0PaPC4O6IHKCsQ3uX9L&returnType=JSON&outType=2&srchTrprId={}&srchTrprDegr={}&srchTorgId={}'.format(tp,n,dic[tp])

    response = requests.get(url)

    js = response.json()

    base_dic = pd.DataFrame.from_dict([json.loads(js['returnJSON'])['inst_base_info']])
    
    df_base_from16718 = pd.concat([df_base_from16718, base_dic])

100%|██████████| 3242/3242 [10:44<00:00,  5.03it/s]


In [34]:
df_base_total = pd.concat([df_base,df_base_from16718])

In [67]:
# 컬럼명 변경

df_base_total.rename(columns = {
                        'trprId': '훈련과정 ID',
                        'trprDegr': '훈련과정 순차',
                        'trprGbn' : '훈련과정 구분',
                        'trprTarget': '주요 훈련과정 구분',
                        'trprTargetNm': '주요 훈련과정 구분명',
                        'trprNm': '훈련과정명',
                        'inoNm' : '훈련기관명',                        
                        'instIno': '훈련기관코드',
                        'traingMthCd': '훈련방법코드',
                        'trprChap': '담당자명',
                        'trprChapTel': '담당자 전화번호',
                        'trprChapEmail': '담당자 이메일',
                        'ncsYn': 'NCS 여부',
                        'ncsCd': 'NCS 코드',
                        'ncsNm': 'NCS 명',
                        'trDcnt': '총 훈련일수',
                        'trtm': '총 훈련시간',
                        'nonNcsCourseTheoryTime': '비 NCS교과 이론시간',
                        'nonNcsCoursePrcttqTime': '비 NCS교과 실기시간',
                        'zipCd': '우편번호',
                        'addr1': '주소지',
                        'addr2': '상세주소',
                        'hpAddr': '홈페이지 주소',
                        'filePath': '파일경로',
                        'pFileName': '로고 파일명',
                        'torgParGrad': '평가등급',
                        'perTrco': '정부지원금',
                        'instPerTrco': '실제 훈련비',
                        'trprUpYn': '??여부'
                        }, inplace = True)

In [69]:
df_base_total.to_csv('base_info.csv')